# Grain Process

In [5]:
import yt.units as u
import numpy as np
atoms = ['H', 'C', 'O']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994}

## Grain model

In [2]:
r = 1000 * u.angstrom
V = 4 / 3 * np.pi * r**3
rho = 3 * u.gram / u.cm**3
Ns = 1e6
G_M_ratio = 100
nH = 2e4 / u.cm**3
T = 10 * u.Kelvin
ns = nH * u.mass_hydrogen / G_M_ratio / rho / V

## Accretion

$$R_{\mathrm{acc}}(i)=\overline{\sigma_{d}n_{d}}\langle v(i)\rangle n(i) = \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)$$

$$k_i= \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)=6.0557\sqrt{\frac{T\ (\text{K})}{\mu}}\times10^{-14}\text{ s}^{-1}$$

In [13]:
def acc(spe, label=0):
    t = ''
    m = 0
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
            m += mass[t]
    react = '{},{},{},NONE,NONE,6.0557e-14*sqrt(Tgas/{})'.format(
        label, spe, grain_spe, m)
    return(react)

In [14]:
LIST_acc = []

'0,H2O,H_dustH_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/18.0152)'